This notebook scrapes 2022-2023 season data weekly and outputs it as a csv file, and a txt file documenting the date and number of games.

In [57]:
# Import packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import numpy as np
from IPython.display import clear_output
from time import sleep

# Set directory path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE THAT YOU CHANGE THE WEEK VARIABLE BELOW')

MAKE SURE THAT YOU CHANGE THE WEEK VARIABLE BELOW


In [58]:
scrape_year = 2023 # change to 2023
# Date and time of scrapping
week = 24 # CHANGE THIS
date1 = datetime.now().strftime('%d/%m')
date2 = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date2 + ', ' + time)

04 April 2023, 01:22:12


In [59]:
# Define function which deals with players having played on multiple teams
def single_player(df):
    if len(df) == 1: # if only one team, keep it
        return df
    else: # multiple teams: keep total (TOT) stats and replace team with last played for
        row = df[df['Tm'] == 'TOT'].copy()
        row['Tm'] = str(df['Tm'].iloc[-1])
        return row
        
# Team abbreviation dictionary
team_dic = {'Sacramento Kings': 'SAC', 'Los Angeles Clippers': 'LAC','Washington Wizards': 'WAS','Phoenix Suns': 'PHO',
   'Boston Celtics': 'BOS','Golden State Warriors': 'GSW','Denver Nuggets': 'DEN','Orlando Magic': 'ORL','Chicago Bulls': 'CHI',
   'Utah Jazz': 'UTA','Toronto Raptors': 'TOR','Los Angeles Lakers': 'LAL','Portland Trail Blazers': 'POR',
   'Memphis Grizzlies': 'MEM','Miami Heat': 'MIA','Atlanta Hawks': 'ATL','Oklahoma City Thunder': 'OKC','Milwaukee Bucks': 'MIL',
   'San Antonio Spurs': 'SAS','Charlotte Hornets': 'CHO','Brooklyn Nets': 'BRK','Cleveland Cavaliers': 'CLE','Philadelphia 76ers': 'PHI',
   'Houston Rockets': 'HOU','Dallas Mavericks': 'DAL','New York Knicks': 'NYK','Indiana Pacers': 'IND','Detroit Pistons': 'DET',
   'Minnesota Timberwolves': 'MIN','New Orleans Pelicans': 'NOP','Charlotte Hornets\xa0': 'CHO'
           }


# Player stats

In [60]:
# Scrape player per game and advanced stats

dfs = [] 
for Type in ['per_game', 'advanced']: # Loop twice: per game, advanced stats

    # Scrape webpage table
    url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_{Type}.html' 
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html')
    table = soup.find('table')
    # Transform table into DataFrame
    stats = pd.read_html(str(table))[0]
    stats = stats[stats['Rk'] != 'Rk'].reset_index(drop = True)
    del stats['Rk'] # Drop rank column

    dfs.append(stats) # Store both DataFrames

# Find unique columns, deal with repeating columns
col_pg = list(dfs[0]) 
col_pg.remove('Player'); col_pg.remove('Tm') # Remove merge columns (Player & team)
temp = list(dfs[1])
unique_cols = [x for x in temp if x not in col_pg] # Keep advanced columns if they are not already in per game columns

# Merge per game and advanced data
data = pd.merge(dfs[0], dfs[1][unique_cols], on = ['Player', 'Tm'])
data['Year'] = len(data) * [scrape_year] # Add year
del data['Unnamed: 19']
del data['Unnamed: 24']

data['Player'] = data['Player'].str.replace('*', '', regex = False) # Delete * in some names
data = data.groupby(['Player']).apply(single_player) # Keep one entry for multiple teams players
data = data.reset_index(drop = True)
data = data.apply(pd.to_numeric, errors = 'ignore')
# Calculate projected win shares
for stat in ['OWS', 'DWS', 'WS', 'VORP'] :
    data[stat] = data[stat] + data[stat] / data['G'] * (82 - data['G'])

data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,16.7,1.242424,0.745455,1.987879,0.121,0.0,-0.6,-0.5,0.248485,2023
1,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,20.3,0.630769,0.000000,0.630769,0.086,-0.7,-2.1,-2.8,0.000000,2023
2,AJ Griffin,SF,19,ATL,69,11,19.2,3.4,7.3,0.466,...,17.8,1.544928,0.950725,2.495652,0.075,-0.2,-0.4,-0.7,0.594203,2023
3,Aaron Gordon,PF,27,DEN,66,66,30.2,6.3,11.2,0.564,...,21.2,5.590909,2.609091,8.200000,0.159,2.5,-0.3,2.2,2.609091,2023
4,Aaron Holiday,PG,26,ATL,59,5,13.3,1.5,3.5,0.424,...,13.4,0.416949,0.833898,1.250847,0.053,-2.9,0.9,-2.0,0.000000,2023


In [61]:
# Srape individual ORtg and DRtg (can only be found in team data)
dfs_rtg = []
for i, tm in enumerate(set(data['Tm'])) :
    
    prog = round(100 * (i+1) / len(set(data['Tm'])), 2) # Compute and print loop progress
    print(f'{prog}% {tm} ...')
    clear_output(wait = True)
    
    # Scrape webpage table
    url = f'https://www.basketball-reference.com/teams/{tm}/{scrape_year}.html'
    page = requests.get(url)
    table = page.text.split(f'id="div_per_poss">')[1].split('</table>')[0] + '</table>'
    soup = BeautifulSoup(table, 'html')
    # Transform table into DataFrame
    data_rtg = pd.read_html(str(soup))[0]
    data_rtg['Tm'] = len(data_rtg) * [tm] # Add team and year
    data_rtg['Year'] = len(data_rtg) * [scrape_year]

    data_rtg = data_rtg.rename(columns = {'Unnamed: 1' : 'Player'})[['Player','Tm','Year','ORtg','DRtg']]
    dfs_rtg.append(data_rtg)

    sleep(2)

# Merge data with individual ORtg and DRtg
data_rtg = pd.concat(dfs_rtg).rename(columns = {'ORtg':'ORtg/100', 'DRtg':'DRtg/100'})
data_rtg['Player'] = data_rtg['Player'].str.replace('*', '', regex = False) # Clean player name column
data = data.merge(data_rtg, on = ['Player','Year','Tm'], how = 'inner') # Merge ratings with data
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Year,ORtg/100,DRtg/100
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,0.745455,1.987879,0.121,0.0,-0.6,-0.5,0.248485,2023,123.0,115
1,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.000000,0.630769,0.086,-0.7,-2.1,-2.8,0.000000,2023,116.0,119
2,AJ Griffin,SF,19,ATL,69,11,19.2,3.4,7.3,0.466,...,0.950725,2.495652,0.075,-0.2,-0.4,-0.7,0.594203,2023,116.0,119
3,Aaron Gordon,PF,27,DEN,66,66,30.2,6.3,11.2,0.564,...,2.609091,8.200000,0.159,2.5,-0.3,2.2,2.609091,2023,124.0,114
4,Aaron Holiday,PG,26,ATL,59,5,13.3,1.5,3.5,0.424,...,0.833898,1.250847,0.053,-2.9,0.9,-2.0,0.000000,2023,111.0,118


# Team stats

In [62]:
# Scrape team stats
url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}.html'
page = requests.get(url)

soup = BeautifulSoup(page.content, 'html')

while soup.find('tr', class_ = 'thead') is not None:
    soup.find('tr', class_ = 'thead').decompose()

table_E = soup.find('table', id = 'divs_standings_E')
teams_E = pd.read_html(str(table_E))[0]
teams_E = teams_E.rename(columns = {'Eastern Conference': 'Team'})

table_W = soup.find('table', id = 'divs_standings_W')
teams_W = pd.read_html(str(table_W))[0]
teams_W = teams_W.rename(columns = {'Western Conference': 'Team'})

teams_E['Seed'] = [len(teams_E[teams_E['W/L%'] > wl])+1 for wl in teams_E['W/L%']]
teams_W['Seed'] = [len(teams_W[teams_W['W/L%'] > wl])+1 for wl in teams_W['W/L%']]

teams = pd.concat([teams_E, teams_W])

del teams['GB']
teams['Team'] = teams['Team'].str.replace('*','', regex = False)
teams['Team'] = teams['Team'].apply(lambda x: ' '.join(x.split()[:-1]))

url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_ratings.html'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html')
soup.find('tr', class_='over_header').decompose()    
table = soup.find('table')
ratings = pd.read_html(str(table))[0][['Team', 'MOV', 'ORtg', 'DRtg', 'NRtg', 'MOV/A', 'ORtg/A', 'DRtg/A', 'NRtg/A']]
ratings['Team'] = ratings['Team'].str.replace('Charlotte Hornets\xa0', 'Charlotte Hornets')
teams['Team'] = teams['Team'].str.replace('Charlotte Hornets\xa0', 'Charlotte Hornets')

teams = teams.merge(ratings, on = 'Team', how = 'outer')
teams = teams.sort_values('W/L%', ascending = False).reset_index(drop = True)
teams['Tm'] = [team_dic.get(tm) for tm in teams['Team']]

# Merge indiviudal and team stats
stats = data.merge(teams, how = 'inner', on = 'Tm')
stats.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,SRS,Seed,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,A.J. Green,SG,23,MIL,33,0,9.7,1.5,3.5,0.440,...,3.9,1,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77
1,Bobby Portis,PF,27,MIL,67,20,25.9,5.6,11.3,0.499,...,3.9,1,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77
2,Brook Lopez,C,34,MIL,76,76,30.3,6.0,11.4,0.528,...,3.9,1,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77
3,Giannis Antetokounmpo,PF,28,MIL,62,62,32.1,11.3,20.4,0.553,...,3.9,1,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77
4,Goran Dragić,PG,36,MIL,55,0,14.7,2.4,5.6,0.421,...,3.9,1,4.0,116.18,112.32,3.85,3.9,116.08,112.3,3.77


In [63]:
# Determine players eligible for ROY and SMOY
url = f'https://www.basketball-reference.com/leagues/NBA_{scrape_year}_rookies.html' # Scrape rookie list
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html')

for overhead in ['over_header', 'over_header thead', 'thead']:
    while soup.find('tr', class_ = overhead) is not None:
        soup.find('tr', class_ = overhead).decompose()
        
table = soup.find('table')
temp = pd.read_html(str(table))[0]
rookies = list(temp['Player'])

stats['roy'] = [1 if player in rookies else 0 for player in stats['Player']] # flag eligible rookies
stats['smoy'] = [1 if gs/g <= 0.5 else 0 for gs, g in zip(stats['GS'], stats['G'])] # flag eligible sixth-mans

for stat in list(stats):
    stats[stat] = stats[stat].replace(np.nan, 0) # replace NaNs with zeros



stats.sort_values('PTS', ascending = False).head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
509,Joel Embiid,C,28,PHI,64,64,34.6,10.9,20.2,0.542,...,4.44,118.96,114.44,4.52,4.44,119.04,114.52,4.52,0,0
31,Luka Dončić,PG,23,DAL,64,64,36.5,11.0,22.2,0.498,...,0.33,117.73,117.44,0.29,0.16,117.70,117.59,0.11,0,0
287,Damian Lillard,PG,32,POR,58,58,36.3,9.6,20.7,0.463,...,-3.21,115.94,119.31,-3.37,-3.13,115.92,119.22,-3.30,0,0
103,Shai Gilgeous-Alexander,PG,24,OKC,66,66,35.5,10.4,20.4,0.510,...,0.87,115.86,115.05,0.81,0.69,115.71,115.09,0.63,0,0
3,Giannis Antetokounmpo,PF,28,MIL,62,62,32.1,11.3,20.4,0.553,...,4.00,116.18,112.32,3.85,3.90,116.08,112.30,3.77,0,0


In [64]:
# Export data to csv
stats.to_csv(path + f'Algorithm/weekly data/week_{week}.csv', index = None)

# Export weekly info to txt
n = teams['W'].sum()
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'w+') as file:
    file.write(f'Date : {date1}\n')
    file.write(f'Date : {date2}\n')
    file.write(f'No. games : {n}\n')
    file.write(f'Time : {time}\n')
    file.close()

In [65]:
print(stats.shape)

(531, 70)
